In [ ]:
pip install selenium pandas

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import os
import time
import re

# Initialize the chrome webdriver
driver = webdriver.Chrome()

# Starting URL - browse by year page
START_URL = 'https://misq.umn.edu/misq/issue/browse-by-year'

# Years to scrape (2010 to 2025)
START_YEAR = 2010
END_YEAR = 2025

# Save CSV file in the same directory as this notebook (MIS_Quarterly folder)
OUT_FILE = os.path.join(os.getcwd(), 'MISQ_Issues.csv')
print(f"CSV file will be saved to: {OUT_FILE}")
print(f"Current working directory: {os.getcwd()}\n")
data = []

def write_to_csv(rows):
    file_exists = os.path.exists(OUT_FILE) and os.path.getsize(OUT_FILE) > 0
    with open(OUT_FILE, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["Title", "URL", "Volume Issue", "Vol Issue Year"])
            print(f"Created CSV file: {OUT_FILE}")
        writer.writerows(rows)
        file.flush()  # Ensure data is written immediately
    print(f"  ✓ Saved {len(rows)} articles to {OUT_FILE}")

def scrape_issue_page(issue_url, vol_issue, year):
    """Scrape articles from a single issue page"""
    try:
        driver.get(issue_url)
        driver.implicitly_wait(15)
        time.sleep(2)
        
        print(f"  Page loaded: {driver.title}")
        
        # Find all article links - try multiple selectors
        articles = []
        
        # Try different selectors for article links
        selectors = [
            'a[href*="/misq/vol"]',
            'a[href*="/article"]',
            '.article-title a',
            '.article a',
            'h3 a',
            'h4 a',
            '.title a',
            'article a',
            '.entry-title a',
            'li a[href*="/article"]',
            'div.article a',
            'table a[href*="/article"]'
        ]
        
        print("  Searching for articles...")
        for selector in selectors:
            try:
                found = driver.find_elements(By.CSS_SELECTOR, selector)
                if found:
                    print(f"    Selector '{selector}': Found {len(found)} links")
                    # Filter for article links (not issue links)
                    filtered = [a for a in found if a.get_attribute('href') and 
                               ('/article' in a.get_attribute('href') or 
                                ('/misq/vol' in a.get_attribute('href') and '/issue' not in a.get_attribute('href')))]
                    if filtered:
                        print(f"      → {len(filtered)} are article links")
                        articles.extend(filtered)
            except Exception as e:
                continue
        
        # Remove duplicates
        seen_urls = set()
        unique_articles = []
        for article in articles:
            try:
                url = article.get_attribute('href')
                if url and url not in seen_urls:
                    seen_urls.add(url)
                    unique_articles.append(article)
            except:
                continue
        
        if not unique_articles:
            # Fallback: find all links and filter
            print("  Trying fallback: checking all links...")
            all_links = driver.find_elements(By.TAG_NAME, 'a')
            for link in all_links:
                try:
                    url = link.get_attribute('href') or ''
                    if url and ('/article' in url or ('/misq/vol' in url and '/issue' not in url)) and url not in seen_urls:
                        seen_urls.add(url)
                        unique_articles.append(link)
                except:
                    continue
        
        print(f"  Total unique articles found: {len(unique_articles)}")
        
        rows = []
        for article in unique_articles:
            try:
                article_url = article.get_attribute('href')
                if not article_url:
                    continue
                    
                # Make sure URL is absolute
                if article_url.startswith('/'):
                    article_url = 'https://misq.umn.edu' + article_url
                
                if not article_url.startswith('http'):
                    continue
                
                # Get article title
                article_title = article.text.strip()
                if not article_title or len(article_title) < 5:
                    # Try to get title from parent or nearby element
                    try:
                        parent = article.find_element(By.XPATH, './..')
                        article_title = parent.text.strip()
                    except:
                        try:
                            # Try sibling or nearby heading
                            heading = article.find_element(By.XPATH, './preceding-sibling::h3 | ./preceding-sibling::h4 | ./following-sibling::h3 | ./following-sibling::h4')
                            article_title = heading.text.strip()
                        except:
                            article_title = "N/A"
                
                if article_url and article_title and article_title != "N/A" and len(article_title) > 5:
                    rows.append([article_title, article_url, vol_issue, year])
                    print(f"    ✓ {article_title[:60]}...")
                    
            except Exception as e:
                print(f"    Error extracting article: {e}")
                continue
        
        if rows:
            write_to_csv(rows)
            return len(rows)
        else:
            print(f"  ⚠ No articles found on this page")
            # Debug: show page structure
            try:
                page_text = driver.find_element(By.TAG_NAME, 'body').text[:300]
                print(f"  Page content preview: {page_text[:200]}...")
            except:
                pass
            return 0
            
    except Exception as e:
        print(f"  ✗ Error scraping issue page: {e}")
        import traceback
        traceback.print_exc()
        return 0

def scrape_year_page(year_url, year):
    """Scrape all issues from a year page"""
    try:
        print(f"\n{'='*60}")
        print(f"Navigating to year page: {year_url}")
        driver.get(year_url)
        driver.implicitly_wait(15)
        time.sleep(3)  # Give page more time to load
        
        print(f"Page title: {driver.title}")
        print(f"Current URL: {driver.current_url}")
        
        # Debug: Print some page content to understand structure
        try:
            page_text = driver.find_element(By.TAG_NAME, 'body').text[:500]
            print(f"Page content preview: {page_text}...")
        except:
            pass
        
        # Find all issue links - try comprehensive approach
        issue_links = []
        
        # Try different selectors for issue links
        selectors = [
            'a[href*="/misq/vol"]',
            'a[href*="/vol"]',
            '.issue-link a',
            '.issue a',
            'h2 a',
            'h3 a',
            'h4 a',
            'li a',
            '.volume a',
            'article a',
            'div a[href*="/vol"]',
            'table a[href*="/vol"]'
        ]
        
        print("\nTrying to find issue links...")
        for selector in selectors:
            try:
                links = driver.find_elements(By.CSS_SELECTOR, selector)
                if links:
                    print(f"  Selector '{selector}': Found {len(links)} links")
                    # Filter for actual issue links
                    filtered = [l for l in links if l.get_attribute('href') and ('/misq/vol' in l.get_attribute('href') or '/vol' in l.get_attribute('href'))]
                    if filtered:
                        print(f"    → {len(filtered)} are issue links")
                        issue_links.extend(filtered)
            except Exception as e:
                print(f"  Selector '{selector}': Error - {e}")
                continue
        
        # Remove duplicates
        seen_urls = set()
        unique_issue_links = []
        for link in issue_links:
            try:
                url = link.get_attribute('href')
                if url and url not in seen_urls:
                    seen_urls.add(url)
                    unique_issue_links.append(link)
            except:
                continue
        
        print(f"\nTotal unique issue links found: {len(unique_issue_links)}")
        
        if not unique_issue_links:
            # Fallback: find ALL links and filter
            print("Trying fallback: checking all links on page...")
            all_links = driver.find_elements(By.TAG_NAME, 'a')
            print(f"Total links on page: {len(all_links)}")
            
            for link in all_links[:50]:  # Check first 50 links
                try:
                    url = link.get_attribute('href') or ''
                    text = link.text.strip()
                    if url and ('/misq/vol' in url or '/vol' in url) and url not in seen_urls:
                        print(f"  Found issue link: {text[:50]} -> {url}")
                        seen_urls.add(url)
                        unique_issue_links.append(link)
                except:
                    continue
        
        # Extract unique issue URLs with metadata
        unique_issues = {}
        for link in unique_issue_links:
            try:
                url = link.get_attribute('href')
                if not url:
                    continue
                    
                # Make sure URL is absolute
                if url.startswith('/'):
                    url = 'https://misq.umn.edu' + url
                
                # Extract volume/issue from URL or text
                link_text = link.text.strip()
                
                # Try to extract from URL
                match = re.search(r'vol[^\d]*(\d+)[^\d]*issue[^\d]*(\d+)', url, re.I)
                if match:
                    vol_issue = f"Vol {match.group(1)}, Issue {match.group(2)}"
                elif link_text and len(link_text) > 3:
                    vol_issue = link_text
                else:
                    # Extract from URL path
                    parts = url.split('/')
                    vol_issue = parts[-1] if parts else f"Vol {year}"
                
                unique_issues[url] = vol_issue
                print(f"  Issue: {vol_issue} -> {url}")
            except Exception as e:
                print(f"  Error processing link: {e}")
                continue
        
        print(f"\n{'='*60}")
        print(f"Processing {len(unique_issues)} issues for year {year}...")
        print(f"{'='*60}")
        
        if len(unique_issues) == 0:
            print(f"WARNING: No issues found for year {year}!")
            print("Page HTML snippet:")
            try:
                html_snippet = driver.page_source[:1000]
                print(html_snippet)
            except:
                pass
            return 0
        
        total_articles = 0
        for issue_url, vol_issue in unique_issues.items():
            print(f"\n{'─'*60}")
            print(f"Scraping: {vol_issue}")
            print(f"URL: {issue_url}")
            count = scrape_issue_page(issue_url, vol_issue, str(year))
            total_articles += count
            print(f"  → Found {count} articles")
            time.sleep(1)  # Be respectful
        
        return total_articles
        
    except Exception as e:
        print(f"Error scraping year page {year_url}: {e}")
        import traceback
        traceback.print_exc()
        return 0

# Main scraping logic
try:
    driver.get(START_URL)
    driver.implicitly_wait(15)
    time.sleep(2)
    
    print("Starting MIS Quarterly scraper (2010-2025)...")
    print(f"Browse page: {START_URL}\n")
    
    # Find year links for 2010-2025
    year_links = {}
    
    print("Searching for year links on browse-by-year page...")
    print(f"Page title: {driver.title}")
    print(f"Current URL: {driver.current_url}\n")
    
    # Get all links and filter by year
    all_links = driver.find_elements(By.TAG_NAME, 'a')
    print(f"Total links found on page: {len(all_links)}")
    
    # First pass: look for links with year in URL or text
    for link in all_links:
        try:
            url = link.get_attribute('href') or ''
            text = link.text.strip()
            
            # Make URL absolute if relative
            if url.startswith('/'):
                url = 'https://misq.umn.edu' + url
            
            # Check if link contains year in URL or text
            for year in range(START_YEAR, END_YEAR + 1):
                year_str = str(year)
                if (year_str in url or year_str in text) and url.startswith('http'):
                    if year not in year_links:
                        year_links[year] = url
                        print(f"  ✓ Found year {year}: {text[:40]} -> {url}")
        except Exception as e:
            continue
    
    print(f"\nFound {len(year_links)} year links directly from page")
    
    # If we didn't find enough year links, try to construct URLs
    if len(year_links) < (END_YEAR - START_YEAR + 1) / 2:  # If less than half found
        print("\nTrying to construct year URLs...")
        # Common patterns for year pages
        base_patterns = [
            'https://misq.umn.edu/misq/issue/browse-by-year/{}',
            'https://misq.umn.edu/misq/issue/{}',
            'https://misq.umn.edu/misq/vol/{}'
        ]
        
        for year in range(START_YEAR, END_YEAR + 1):
            if year in year_links:
                continue  # Skip if already found
                
            for pattern in base_patterns:
                test_url = pattern.format(year)
                try:
                    driver.get(test_url)
                    time.sleep(1)
                    if '404' not in driver.title.lower() and 'not found' not in driver.title.lower():
                        year_links[year] = test_url
                        print(f"  ✓ Constructed year {year}: {test_url}")
                        break
                except:
                    continue
    
    # Go back to browse page
    driver.get(START_URL)
    time.sleep(2)
    
    print(f"\n{'='*60}")
    print(f"Total year links found: {len(year_links)}")
    print(f"Years: {sorted(year_links.keys())}")
    print(f"{'='*60}\n")
    
    # Scrape each year
    total_articles_scraped = 0
    for year in sorted(year_links.keys()):
        if START_YEAR <= year <= END_YEAR:
            print(f"\n{'='*60}")
            print(f"Scraping year {year}")
            print(f"{'='*60}")
            count = scrape_year_page(year_links[year], year)
            total_articles_scraped += count
            print(f"Year {year}: {count} articles scraped")
            time.sleep(2)
    
    print(f"\n{'='*60}")
    print(f"Scraping complete!")
    print(f"Total articles scraped: {total_articles_scraped}")
    print(f"{'='*60}")

except Exception as e:
    print(f"Exception: {e}")
    import traceback
    traceback.print_exc()

finally:
    driver.quit()

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv, time, os, random, re
from bs4 import BeautifulSoup
from datetime import datetime

START_INDEX = 245
END_INDEX = 300
WAIT_SEC = 15

CSV_PATH = os.path.join(os.getcwd(), "MISQ_Issues.csv")
OUT_FILE = os.path.join(os.getcwd(), "MISQ_article_data.csv")
JOURNAL_TITLE = "MIS Quarterly"

def clean_text(s):
    if not s:
        return "N/A"
    s = re.sub(r"\s+", " ", str(s).strip())
    return s if s else "N/A"

def parse_month_year(date_str):
    if not date_str:
        return "N/A"
    date_str = date_str.strip()
    for fmt in ("%Y/%m/%d", "%Y-%m-%d", "%b %d, %Y", "%B %d, %Y"):
        try:
            d = datetime.strptime(date_str, fmt)
            return d.strftime("%B %Y")
        except:
            pass
    return clean_text(date_str)

def get_driver():
    opts = Options()

    # Windows-safe persistent profile path (in your repo folder)
    profile_dir = os.path.join(os.getcwd(), "chrome_profile_misq")
    opts.add_argument(f"--user-data-dir={profile_dir}")

    # IMPORTANT: allow images so captcha/verification can render
    prefs = {"profile.managed_default_content_settings.images": 1}
    opts.add_experimental_option("prefs", prefs)

    return webdriver.Chrome(options=opts)

def extract_article_details(html):
    soup = BeautifulSoup(html, "html.parser")
    meta = {}
    for m in soup.find_all("meta"):
        n = m.get("name")
        c = m.get("content")
        if n and c:
            meta[n.strip()] = c.strip()

    title = clean_text(meta.get("citation_title"))
    if title == "N/A":
        h1 = soup.select_one("h1")
        title = clean_text(h1.get_text()) if h1 else "N/A"

    abstract = "N/A"
    if meta.get("citation_abstract"):
        abstract = clean_text(meta.get("citation_abstract"))
    else:
        abs_el = soup.select_one(".abstract, .abstract-text, #abstract, .article-abstract")
        if abs_el:
            abstract = clean_text(abs_el.get_text())

    kw = []
    for k in soup.find_all("meta", attrs={"name": "citation_keywords"}):
        if k.get("content"):
            kw.append(clean_text(k.get("content")))
    keywords = clean_text(str(sorted(set([x for x in kw if x and x != "N/A"]))))

    vol = clean_text(meta.get("citation_volume"))
    issue = clean_text(meta.get("citation_issue"))
    volume_issue = "N/A"
    if vol != "N/A" and issue != "N/A":
        volume_issue = f"Volume {vol}, Issue {issue}"
    elif vol != "N/A":
        volume_issue = f"Volume {vol}"
    elif issue != "N/A":
        volume_issue = f"Issue {issue}"

    pub_date = meta.get("citation_publication_date") or meta.get("citation_date")
    month_year = parse_month_year(pub_date)

    return title, abstract, keywords, volume_issue, month_year

def extract_authors_hover(driver):
    def _extract_email(text):
        if not text:
            return "N/A"
        m = re.search(r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}", text, flags=re.I)
        return clean_text(m.group(0)) if m else "N/A"

    def _dedupe(items):
        uniq, seen = [], set()
        for a in items:
            key = (a.get("name"), a.get("email"), a.get("address"))
            if key in seen:
                continue
            seen.add(key)
            uniq.append(a)
        return uniq

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    meta_authors = [m.get("content", "").strip() for m in soup.select('meta[name="citation_author"]') if m.get("content")]
    meta_insts = [m.get("content", "").strip() for m in soup.select('meta[name="citation_author_institution"]') if m.get("content")]
    meta_emails = [m.get("content", "").strip() for m in soup.select('meta[name="citation_author_email"]') if m.get("content")]

    if meta_authors and (meta_insts or meta_emails):
        out = []
        for i, name in enumerate(meta_authors):
            inst = meta_insts[i] if i < len(meta_insts) else "N/A"
            em = meta_emails[i] if i < len(meta_emails) else "N/A"
            out.append({"name": clean_text(name), "email": clean_text(em), "address": clean_text(inst)})
        return _dedupe([a for a in out if a["name"] != "N/A"])

    actions = ActionChains(driver)

    author_els = driver.find_elements(
        By.CSS_SELECTOR,
        "a[href*='/misq/author/'], .article-authors a, .authors a, a.article-author"
    )
    if not author_els:
        return []

    tip_selectors = [
        "[role='tooltip']",
        ".tooltipster-base",
        ".tooltipster-content",
        ".tippy-box",
        ".tippy-content",
        ".ui-tooltip",
        ".tooltip",
        ".popover",
        ".popover-body",
    ]

    def _get_visible_tooltip_text():
        for sel in tip_selectors:
            tips = driver.find_elements(By.CSS_SELECTOR, sel)
            vis = [t for t in tips if t.is_displayed() and (t.text or "").strip()]
            if vis:
                return vis[-1].text.strip()
        return ""

    results = []
    for el in author_els:
        name = clean_text(el.text)
        if name == "N/A":
            continue

        email = "N/A"
        address = "N/A"

        try:
            href = el.get_attribute("href") or ""
            if "mailto:" in href:
                email = clean_text(href.replace("mailto:", "").split("?")[0])
        except:
            pass

        trigger = None
        try:
            parent = el.find_element(By.XPATH, "./parent::*")
            cand = parent.find_elements(
                By.CSS_SELECTOR,
                "button[data-toggle='tooltip'], button[data-bs-toggle='tooltip'], [data-tippy-content], [data-original-title], a[title], button[title], span[title]"
            )
            trigger = cand[0] if cand else None
        except:
            trigger = None

        if trigger is None:
            try:
                gp = el.find_element(By.XPATH, "./ancestor::*[self::li or self::div or self::span][1]")
                cand = gp.find_elements(By.CSS_SELECTOR, "button, a, span, i, svg")
                for c in cand:
                    attrs = [
                        c.get_attribute("data-tippy-content"),
                        c.get_attribute("data-original-title"),
                        c.get_attribute("title"),
                        c.get_attribute("aria-label"),
                    ]
                    if any(x and str(x).strip() for x in attrs):
                        trigger = c
                        break
            except:
                trigger = None

        tip_text = ""

        if trigger is not None:
            for attr in ["data-tippy-content", "data-original-title", "title", "aria-label"]:
                try:
                    v = trigger.get_attribute(attr)
                    if v and str(v).strip():
                        tip_text = str(v).strip()
                        break
                except:
                    pass

        if not tip_text:
            try:
                if trigger is not None:
                    try:
                        actions.move_to_element(trigger).pause(0.4).perform()
                    except:
                        actions.move_to_element(el).pause(0.4).perform()
                    try:
                        trigger.click()
                        time.sleep(0.2)
                    except:
                        pass
                else:
                    actions.move_to_element(el).pause(0.4).perform()

                WebDriverWait(driver, 2).until(lambda d: bool(_get_visible_tooltip_text()))
                tip_text = _get_visible_tooltip_text()
            except:
                tip_text = _get_visible_tooltip_text()

        if tip_text:
            if "<" in tip_text and ">" in tip_text:
                try:
                    tip_text = BeautifulSoup(tip_text, "html.parser").get_text("\n")
                except:
                    pass

            lines = [x.strip() for x in tip_text.split("\n") if x.strip()]
            if lines and lines[0].lower() == name.lower():
                lines = lines[1:]

            joined = " ".join(lines)
            found_email = _extract_email(joined)
            if email == "N/A" and found_email != "N/A":
                email = found_email

            joined = re.sub(r"\bSearch for other works\b.*", "", joined, flags=re.I).strip()
            if found_email != "N/A":
                joined = re.sub(re.escape(found_email), "", joined, flags=re.I).strip(" -|,;\t\n")

            address = clean_text(joined) if joined else address

        results.append({"name": name, "email": email, "address": address})

    return _dedupe(results)

if not os.path.exists(CSV_PATH):
    print(f"Error: {CSV_PATH} not found.")
    raise SystemExit

df = pd.read_csv(CSV_PATH)

if not os.path.exists(OUT_FILE):
    with open(OUT_FILE, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow([
            "URL",
            "Journal_Title",
            "Article_Title",
            "Volume_Issue",
            "Month_Year",
            "Abstract",
            "Keywords",
            "Author_name",
            "Author_email",
            "Author_Address"
        ])

driver = get_driver()
wait = WebDriverWait(driver, WAIT_SEC)

try:
    for i in range(START_INDEX, min(END_INDEX, len(df))):
        row = df.iloc[i]
        url = str(row.get("URL", "")).strip()
        if not url:
            continue

        print(f"[{i+1}/{min(END_INDEX, len(df))}] {url}")
        driver.get(url)

        time.sleep(random.uniform(8, 15))

        def _looks_like_cloudflare_challenge(d):
            try:
                u = (d.current_url or "").lower()
                t = (d.title or "").lower()
                if "/cdn-cgi/" in u:
                    return True
                if "just a moment" in t:
                    return True
                src = (d.page_source or "").lower()
                # Common Cloudflare / Turnstile markers
                if "cf-turnstile" in src:
                    return True
                if "challenge-platform" in src:
                    return True
                if "cf_chl" in src:
                    return True
            except:
                return False
            return False

        # If verification shows up, let you solve it and continue automatically
        if _looks_like_cloudflare_challenge(driver):
            print("Verification detected. Solve it in the opened browser window; I'll continue once the article loads.")
            try:
                WebDriverWait(driver, 300).until(lambda d: not _looks_like_cloudflare_challenge(d))
            except:
                pass

        # Proceed once the real article page is present
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1")))

        html = driver.page_source
        article_title, abstract, keywords, volume_issue, month_year = extract_article_details(html)

        authors = extract_authors_hover(driver)
        if not authors:
            authors = [{"name": "N/A", "email": "N/A", "address": "N/A"}]

        with open(OUT_FILE, "a", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            for a in authors:
                w.writerow([
                    url,
                    JOURNAL_TITLE,
                    article_title,
                    volume_issue,
                    month_year,
                    abstract,
                    keywords,
                    a["name"],
                    a["email"],
                    a["address"]
                ])

        print(f"   Saved: {len(authors)} author rows")

finally:
    driver.quit()
    print(f"\nDone. Output saved to: {OUT_FILE}")

[246/300] https://misq.umn.edu/misq/article/37/2/617/105/Digital-Business-Strategy-and-Value-Creation
   Saved: 1 author rows
[247/300] https://misq.umn.edu/misq/article/37/2/633/110/Visions-and-Voices-on-Emerging-Challenges-in
   Saved: 4 author rows
[248/300] https://misq.umn.edu/misq/article/doi/10.25300/MISQ/2026/18511/3786/Trustworthiness-in-Computational-Theory
   Saved: 3 author rows
[249/300] https://misq.umn.edu/misq/article/doi/10.25300/MISQ/2026/18708/3771/When-Bots-Evaluate-Humans-Delegation-to-Bots-and

Done. Output saved to: /Users/keerthisagi/Documents/Journals/MIS Quarterly/MISQ_article_data.csv


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=145.0.7632.117)
Stacktrace:
0   chromedriver                        0x00000001046a96b4 cxxbridge1$str$ptr + 3127600
1   chromedriver                        0x00000001046a1a50 cxxbridge1$str$ptr + 3095756
2   chromedriver                        0x000000010417e56c _RNvCsdExgN8vFLbb_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 75432
3   chromedriver                        0x0000000104156eb4 chromedriver + 159412
4   chromedriver                        0x00000001041f0274 _RNvCsdExgN8vFLbb_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 541616
5   chromedriver                        0x0000000104206148 _RNvCsdExgN8vFLbb_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 631428
6   chromedriver                        0x00000001041bbb9c _RNvCsdExgN8vFLbb_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 326872
7   chromedriver                        